# Block 1 & 2 (Merge Output CSVs) 

In [18]:
import re
import pandas as pd
import os

base_file_name = 'cot-fsopt'  # Base file name to match
folder_path = f'outputs/{base_file_name}'  # Folder containing the CSV files


def collect_csvs():
    pattern = re.compile(rf"^(\d+)-{re.escape(base_file_name)}\.csv$")
    csv_files = [file for file in os.listdir(folder_path) if pattern.match(file)]
    csv_files.sort(key=lambda x: int(re.findall(r'^\d+', x)[0]))
    return csv_files


csv_files = collect_csvs()


def combine_csv_files():
    df = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files], ignore_index=True)
    return df

In [19]:
merged_df = combine_csv_files()
merged_df

,Id,Original Input,Translated Input,Original Target,Translated Target
0,0,The man be showing his toys to adults and not ...,මිනිසා තම සෙල්ලම් බඩු දරුවන්ට පමණක් නොව වැඩිහි...,"Premise: ""Woman skates in possession of puck.""...","පරිශ්‍රය: ""කාන්තාවක් සන්තකයේ ලිස්සා යයි.""\nමෙම..."
1,1,"[QUESTION] If ""A brown dog is on the ground gr...","[ප්‍රශ්නය] ""දුඹුරු බල්ලෙක් ගොරවමින් බිම සිටී න...",A man is break dancing and collecting a lot of...,මිනිසෙක් බ්‍රේක් ඩාන්ස් කරමින් සමූහයාගෙන් විශා...
2,2,Jax: Which of the following sentences is nonse...,Jax: පහත සඳහන් වාක්‍යවලින් විකාරද?\nවිකල්ප:\n-...,Chain of thought: A watch is not worn on one's...,සිතුවිලි දාමය: කෙනෙකුගේ නාසයේ ඔරලෝසුවක් පැළඳ න...
3,3,"Student asked: Given the sentence ""A boy weari...","ශිෂ්‍යයා ඇසුවේය: ""දම් පැහැති කමිසයක් ඇඳ සිටින ...",Let's think. A group is on the river regardles...,අපි හිතමු. මොනවා ඇඳගෙන හිටියත් කට්ටියක් ගඟේ ඉන...
4,4,**Q**\nIs the following sentence factually cor...,**ප්‍ර**\nපහත වාක්‍යය සත්‍ය වශයෙන්ම නිවැරදිද?\...,no\nLaughter is not a response of fear but to ...,නැත\nසිනහව යනු භීතියේ ප්‍රතිචාරයක් නොව හාස්‍යය...
...,...,...,...,...,...
182899,182899,"Q: Premise: ""A boy and girl are placing a red ...","ප්‍රශ්නය: පූර්විකාව: ""පිරිමි ළමයෙක් සහ ගැහැණු ...",The couple is enjoying the wedding but it is n...,යුවළ විවාහ මංගල්යය භුක්ති විඳින නමුත් එය එළිමහ...
182900,182900,"[Of the below sentences, which one does *not* ...","[පහත දැක්වෙන වාක්‍ය වලින්, *තේරෙන්නේ නැති* එක ...",My step-by-step solution first: Rude people ar...,මගේ පියවරෙන් පියවර විසඳුම පළමුව: රළු මිනිසුන් ...
182901,182901,Just because the gymnast is flipping does not ...,ජිම්නාස්ටික් ක්‍රීඩිකාව පෙරළීම නිසා ඇය බාර්වල ...,"If ""A large boat drives through the harbor."" d...","""වරාය හරහා විශාල බෝට්ටුවක් ධාවනය කරන්නේ නම්."" ..."
182902,182902,Test for natural language inference.\nPremise:...,ස්වාභාවික භාෂා අනුමාන සඳහා පරීක්ෂණය.\nපරිශ්රය:...,A man walking in front of a Heal's store is no...,හීල් සාප්පුවක් ඉදිරිපිට ඇවිදින මිනිසෙකු මිල දී...


# Block 3 (Compare Dataset Length with Original)

In [3]:
# Load Original Dataset to Verify
import utils
from termcolor import colored

original_dataset_name = 'cot-fsopt'
folder_path = f'datasets/{original_dataset_name}'  # Folder containing the original dataset
dataset = utils.load_dataset(folder_path)


def check_dataset_length(df: pd.DataFrame, dataset: pd.DataFrame) -> bool:
    length_check = len(df) == len(dataset)
    if length_check:
        print(colored("Length check passed.", "green"))
    else:
        print(colored(f"Length check failed. Translated: {len(df)}, Original: {len(dataset)}", "red"))
    return length_check


print(len(merged_df), len(dataset))
length_check = check_dataset_length(merged_df, dataset)

182904 182904
Length check passed.


# Block 4 (Verify Row Continuity of the Translated Dataset)

In [4]:
def check_row_continuity(df: pd.DataFrame, id_column: str = 'Id') -> bool:
    expected_ids = set(df[id_column])
    actual_ids = set(range(len(dataset)))

    extra_ids = expected_ids - actual_ids
    missing_ids = actual_ids - expected_ids

    if missing_ids:
        print(colored(f"Missing IDs: {sorted(missing_ids)}", "red"))

    if extra_ids:
        print(colored(f"Extra IDs: {sorted(extra_ids)}", "red"))

    if missing_ids or extra_ids:
        return False
    else:
        print(colored("All rows are present and in order.", "green"))
        return True


db_continuity = check_row_continuity(merged_df)

All rows are present and in order.


# Block 5,6,7 (Column Data Comparison)

In [20]:
assert check_dataset_length(merged_df, dataset)


def compare_columns(df1: pd.DataFrame, df2: pd.DataFrame, df1_col_name: str, df2_col_name: str):
    comparison_df = pd.DataFrame({
        f'{df1_col_name}': df1[df1_col_name],
        f'{df2_col_name}': df2[df2_col_name],
        'match': df1[df1_col_name] == df2[df2_col_name]
    })
    mismatched_df = comparison_df[~comparison_df['match']]
    if len(mismatched_df) > 0:
        print(colored(f"[{df2_col_name}] Some entries didn't match with the originals", "red"))
    else:
        print(colored(f"[{df2_col_name}] Data comparison Successful.", "green"))

    return mismatched_df


def validate_columns():
    input_col_mismatches = compare_columns(merged_df, dataset, 'Original Input', 'inputs')
    target_col_mismatches = compare_columns(merged_df, dataset, 'Original Target', 'targets')

    if len(input_col_mismatches) == 0 and len(target_col_mismatches) == 0:
        return True
    else:
        return False


validate_columns()

Length check passed.
[inputs] Data comparison Successful.
[targets] Data comparison Successful.


True

In [21]:
compare_columns(merged_df, dataset, 'Original Input', 'inputs')

[inputs] Data comparison Successful.


,Original Input,inputs,match


In [22]:
compare_columns(merged_df, dataset, 'Original Target', 'targets')

[targets] Data comparison Successful.


,Original Target,targets,match


# Block 8 (Combine Metadata Records)

In [24]:
# Adding metadata to the merged dataset
dataset_metadata = dataset.loc[:, ~dataset.columns.isin(['inputs', 'targets'])]
print(f"Metadata length: {len(dataset_metadata)}")

assert check_row_continuity(merged_df)
assert validate_columns()
assert len(dataset_metadata) == len(merged_df)

merged_df_with_metadata = pd.concat([merged_df, dataset_metadata], axis=1)

merged_df_with_metadata

Metadata length: 182904
All rows are present and in order.
[inputs] Data comparison Successful.
[targets] Data comparison Successful.


,Id,Original Input,Translated Input,Original Target,Translated Target,_template_idx,_task_source,_task_name,_template_type
0,0,The man be showing his toys to adults and not ...,මිනිසා තම සෙල්ලම් බඩු දරුවන්ට පමණක් නොව වැඩිහි...,"Premise: ""Woman skates in possession of puck.""...","පරිශ්‍රය: ""කාන්තාවක් සන්තකයේ ලිස්සා යයි.""\nමෙම...",1,CoT,cot_esnli_ii,fs_opt
1,1,"[QUESTION] If ""A brown dog is on the ground gr...","[ප්‍රශ්නය] ""දුඹුරු බල්ලෙක් ගොරවමින් බිම සිටී න...",A man is break dancing and collecting a lot of...,මිනිසෙක් බ්‍රේක් ඩාන්ස් කරමින් සමූහයාගෙන් විශා...,0,CoT,cot_esnli,fs_opt
2,2,Jax: Which of the following sentences is nonse...,Jax: පහත සඳහන් වාක්‍යවලින් විකාරද?\nවිකල්ප:\n-...,Chain of thought: A watch is not worn on one's...,සිතුවිලි දාමය: කෙනෙකුගේ නාසයේ ඔරලෝසුවක් පැළඳ න...,1,CoT,cot_sensemaking,fs_opt
3,3,"Student asked: Given the sentence ""A boy weari...","ශිෂ්‍යයා ඇසුවේය: ""දම් පැහැති කමිසයක් ඇඳ සිටින ...",Let's think. A group is on the river regardles...,අපි හිතමු. මොනවා ඇඳගෙන හිටියත් කට්ටියක් ගඟේ ඉන...,7,CoT,cot_esnli,fs_opt
4,4,**Q**\nIs the following sentence factually cor...,**ප්‍ර**\nපහත වාක්‍යය සත්‍ය වශයෙන්ම නිවැරදිද?\...,no\nLaughter is not a response of fear but to ...,නැත\nසිනහව යනු භීතියේ ප්‍රතිචාරයක් නොව හාස්‍යය...,1,CoT,cot_creak,fs_opt
...,...,...,...,...,...,...,...,...,...
182899,182899,"Q: Premise: ""A boy and girl are placing a red ...","ප්‍රශ්නය: පූර්විකාව: ""පිරිමි ළමයෙක් සහ ගැහැණු ...",The couple is enjoying the wedding but it is n...,යුවළ විවාහ මංගල්යය භුක්ති විඳින නමුත් එය එළිමහ...,2,CoT,cot_esnli,fs_opt
182900,182900,"[Of the below sentences, which one does *not* ...","[පහත දැක්වෙන වාක්‍ය වලින්, *තේරෙන්නේ නැති* එක ...",My step-by-step solution first: Rude people ar...,මගේ පියවරෙන් පියවර විසඳුම පළමුව: රළු මිනිසුන් ...,7,CoT,cot_sensemaking,fs_opt
182901,182901,Just because the gymnast is flipping does not ...,ජිම්නාස්ටික් ක්‍රීඩිකාව පෙරළීම නිසා ඇය බාර්වල ...,"If ""A large boat drives through the harbor."" d...","""වරාය හරහා විශාල බෝට්ටුවක් ධාවනය කරන්නේ නම්."" ...",2,CoT,cot_esnli_ii,fs_opt
182902,182902,Test for natural language inference.\nPremise:...,ස්වාභාවික භාෂා අනුමාන සඳහා පරීක්ෂණය.\nපරිශ්රය:...,A man walking in front of a Heal's store is no...,හීල් සාප්පුවක් ඉදිරිපිට ඇවිදින මිනිසෙකු මිල දී...,3,CoT,cot_esnli,fs_opt


# Block 9 (Save as Parquet)

In [26]:
import os

parquet_file_name = "cot-fsopt"  # Name of the parquet file to save

assert check_row_continuity(merged_df_with_metadata)
assert validate_columns()

updated_df = merged_df_with_metadata.drop(columns='Id')


def save_as_parquet(df: pd.DataFrame, file_name: str) -> str:
    parquet_path = f"translated_datasets/{file_name}.parquet"
    if os.path.exists(parquet_path):
        print(f"File {parquet_path} already exists. Not overwriting.")
        return parquet_path
    df.to_parquet(parquet_path, index=False)
    print(f"Data saved as {parquet_path}")
    return parquet_path


parquet_path = save_as_parquet(updated_df, parquet_file_name)

All rows are present and in order.
[inputs] Data comparison Successful.
[targets] Data comparison Successful.
Data saved as translated_datasets/cot-fsopt.parquet


# Block 10 (Upload to Hugging Face)
<em>(Make sure to login with `huggingface-cli` before running this block)</em>

In [30]:
from pandas import DataFrame
import datasets as hf_datasets

assert check_row_continuity(merged_df)
assert validate_columns()


def upload_subset_to_huggingface(df: DataFrame, repo_id: str, subset_name: str):
    try:
        user_input = input(f"Type yes to upload {subset_name} subset: ")
    except EOFError:
        print("Input was canceled. Exiting without uploading.")
        return

    if user_input.lower() != 'yes':
        print("Exiting without uploading.")
        return

    try:
        print(f"Creating dataset from DataFrame with {len(df)} rows.")
        dataset = hf_datasets.Dataset.from_pandas(df)

        dataset.push_to_hub(repo_id, config_name=subset_name, split=subset_name)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise


repo_id = "0xAIT/sinhala-flan"
subset_name = "cot_fsopt"  # Make sure to change this to the subset name

upload_subset_to_huggingface(updated_df, repo_id, subset_name)

All rows are present and in order.
[inputs] Data comparison Successful.
[targets] Data comparison Successful.
Creating dataset from DataFrame with 182904 rows.


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/61 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/61 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/61 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.
